In [1]:
%load_ext autoreload
%autoreload 2

# Non-adiabatic dynamics 
This tutorial shows how to run non-adiabatic dynamics with a trained model using the [Zhu-Nakamura surface hopping method](https://pubs.rsc.org/en/content/articlelanding/2014/cp/c4cp03498h).

First let's import dependencies:


In [2]:
import sys
import os

# so that NFF is in your path
sys.path.insert(0, "..")

import json
from nff.md.zhu_nakamura.dynamics import CombinedZhuNakamura
from ase.io.trajectory import Trajectory
import nglview as nv



Now we'll find a trained model. The trained azobenzene models can be found in `NeuralForceField/models/azo_derivatives`. The sub-folders are for diabatic and adiabatic models, trained either with the full set of geometries, or with 40 species held out. There are also three models trained with different splits and different initialization from random seeds:

In [3]:
print(os.listdir('../models/azo_derivatives'))
print(os.listdir('../models/azo_derivatives/all_diabatic'))

['all_diabatic', 'holdout_diabatic', 'holdout_adiabatic']
['seed_0', 'seed_2', 'seed_1']


We'll use the diabatic model trained on all species, with seed 0: `../models/azo_derivatives/all_diabatic/seed_0`.

## ZN

The script for Tully's surface hopping is `NeuralForceField/nff/md/zhu_nakamura/run_zn`. If you run the script and supply the path of a JSON parameter file, it will do the rest for you. Here we'll go through some parameters to give, and show a little of what goes on behind the scenes.

We'll have to define `ground_params`, `zhu_params`, `combined_params`, and `all_params`. The first is for parameters in the ground state MD simulation, the second for ZN surface hopping, and the third is for batching. The fourth is for some remaining parameters, which we'll explain below.

Let's define `ground_params`:

In [4]:
ground_params = {'ttime': 50,
                 'logfile': 'ground.log',
                 'max_time': 200,
                 'savefile': 'ground.trj',
                 'timestep': 0.5,
                 'equil_time': 100,
                 'thermostat': 'nosehoover',
                 'loginterval': 10,
                 'temperature': 300}

Now let's do `zhu_params` and `batched_params`:

In [5]:
zhu_params = {'log_file': 'trj.log',
 'max_time': 200,
 'out_file': 'trj.csv',
 'timestep': 0.5,
 'num_states': 2,
 'max_gap_hop': 0.021673306772908366,
 'save_period': 5,
 'initial_surf': 1,
 'initial_time': 0.0}

batched_params = {'cutoff': 5.0,
 'device': 1,
 'num_trj': 10,
 'batch_size': 5,
 'nbr_update_period': 10}

Lastly we'll define `all_params`, which has the starting coordinates and the model path:

In [6]:
with open('data/azo_coords.json', 'r') as f:
    coords = json.load(f)

all_params = {"coords": coords, # starting geometry of the molecule
              'model_path': '../models/azo_derivatives/all_diabatic/seed_0',
              "cutoff": 5.0,
              "zhu_params": zhu_params,
              "ground_params": ground_params,
              "batched_params": batched_params
             }

When we run the script from the command line, it parses these three dictionaries from a file and makes an instance of `CombinedZhuNakamura`, like this:

In [7]:
from nff.md.zhu_nakamura.run_zn import coords_to_xyz, make_dataset, make_trj

coords = all_params["coords"]
nxyz = [coords_to_xyz(coords)]


print('loading models')

dataset = make_dataset(nxyz=nxyz, all_params=all_params)

print('running ground state + Zhu-Nakamura dynamics')

zn = make_trj(all_params=all_params,
              dataset=dataset)



loading models
running ground state + Zhu-Nakamura dynamics


/home/saxelrod/miniconda3/envs/nff/lib/python3.7/site-packages/ase/md/md.py:48: FutureWarning: Specify the temperature in K using the 'temperature_K' argument
  warnings.warn(FutureWarning(w))


Now we can run it!

In [8]:
zn.run()


../nff/nn/modules/diabat.py:140: UserWarning: torch.symeig is deprecated in favor of torch.linalg.eigh and will be removed in a future PyTorch release.
The default behavior has changed from using the upper triangular portion of the matrix by default to using the lower triangular portion.
L, _ = torch.symeig(A, upper=upper)
should be replaced with
L = torch.linalg.eigvalsh(A, UPLO='U' if upper else 'L')
and
L, V = torch.symeig(A, eigenvectors=True)
should be replaced with
L, V = torch.linalg.eigh(A, UPLO='U' if upper else 'L') (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448224956/work/aten/src/ATen/native/BatchLinearAlgebra.cpp:2500.)
  ad_energies, u = torch.symeig(d_mat, True)


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 10.76 GiB total capacity; 433.06 MiB already allocated; 8.44 MiB free; 508.00 MiB reserved in total by PyTorch)

We can view the ground-state log file:

In [ ]:
with open('ground.log', 'r') as f:
    ground_log = f.read()
print(ground_log)

We see that all energies fluctuate, as kinetic energy is being added into the system fo the thermostat. The temperature also varies, and over enough time it will average out to 300 K. 

To get the actual geometries, energies, and forces, we can load the trajectory file. And we can visualize it with `nglview`:



In [ ]:
trj = Trajectory('ground.trj')
nv.show_asetraj(trj)

We can also view the excited state log:

In [ ]:
with open('tully.log', 'r') as f:
    tully_log = f.read()
print(tully_log)

This log shows us the population in the different states at each time, the norm of the electronic coefficient vector $c$, and the maximum hopping probability of any of the trajectories. We see that all of the trajectories have returned to the ground state by 200 fs.

To get the geometries, forces, etc., we can use the `NeuralTully` class method `from_pickle`:



In [ ]:
state_dicts, trjs = NeuralTully.from_pickle('tully.pickle')

The output contains `state_dicts`, a set of dictionaries with information about each trajectory for each time step, and `trjs`, a set of ASE trajectories. We can visualize one of the trajectories:

In [ ]:
trj = trjs[0]
nv.show_asetraj(trj)

We can also take a look at `state_dicts`:

In [ ]:
print(len(state_dicts))
print(list(state_dicts[0].keys()))


We see that there is one dictionary for each saved time step (all time steps in which a hop occurs are automatically saved, even if you only asked the geoms to be saved every X steps). Each dictionary has various properties, each of shape `num_trj  x ...`, so that the first dimension divides properties by trajectory. For example:



In [ ]:
state_dict = state_dicts[0]
print('nxyz: ', state_dict['nxyz'].shape) # coordinates 
print('energy: ', state_dict['energy'].shape) # energy
print('forces: ', state_dict['forces'].shape) # forces on each surface
print('H_d: ', state_dict['H_d'].shape) # diabatic Hamiltonian
print('U: ', state_dict['U'].shape) # unitary transformation from diabatic to adiabatic 
print('vel: ', state_dict['vel'].shape) # velocities
print('c: ', state_dict['c'].shape) # c electronic wavefunction vector 
print('surfs: ', state_dict['surfs'].shape) # current surface


print(state_dict['force_nacv']) # not computed because we did diabatic propagation
print(state_dict['T']) # T matrix (NACV off-diagonal components of Hamiltonian)
                       # also not computed because we did diabatic propagation